In [ ]:
from openai import OpenAI
from collections import Counter
from natsort import natsorted
import requests
from requests.exceptions import Timeout, RequestException

model_name = "qwen2.5:72b"
client = OpenAI(
    base_url='http://kt-gpu5.ijs.si:11435/v1',
    api_key='ollama',  # required, but unused
)

def create_chat_completions(model_name, message, timeout=100):
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": message},
            ],
            timeout=timeout
        )
        return response.choices[0].message.content
    except Timeout:
        return None
    except RequestException as e:
        return None

In [ ]:
annotator_message = f"""You are an expert linguist helping to identify multi-word expressions (MWEs) in a large corpus. A multi-word expression is a sequence of words that form a single unit of meaning and cannot be easily deduced by the meanings of individual words.

Here is the information about a potential MWE:

**Candidate Phrase:** "{n-gram}"  
**PMI Score:** {PMI_score}  
**Frequency in Corpus:** {frequency}

### Known MWEs for Reference:
- "{known_mwe_1}"
- "{known_mwe_2}"
- "{known_mwe_3}"

### Example Sentences:
1. "{sentence_1}"
2. "{sentence_2}"
3. "{sentence_3}"

**Questions:**
1. Does the candidate phrase overlap in meaning or structure with any known MWEs? If so, which one(s)?
2. Could this phrase be considered a new variation or extension of an existing MWE? Why?
3. If it is novel, does it demonstrate properties of an MWE such as idiomaticity or collocational fixedness?
4. How likely is it that this expression is becoming a trend in social media language? Rate this likelihood on a scale from 1-5.
5. Based on the given information, would you classify this candidate as:
   - A novel MWE
   - A variation of an existing MWE
   - Not an MWE

Explain your decision with examples and reasoning.
"""

judge_message = f"""You are a judge evaluating the response of a linguist who has classified a multi-word expression (MWE) in a large corpus. Based on that, can you provide a label for the candidate phrase from the following options: ["novel MWE", "variation of an existing MWE", "not an MWE"]? Please provide only the label without any additional information."""